In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

In [2]:
# STEP 1: Load the CSV File
df = pd.read_csv('product_orders.csv')
print("Initial Data Preview:")
print(df.head())

Initial Data Preview:
   order_id product_id       product_name     category  order_date  quantity  \
0     44605   5e09a9ee     Wireless Mouse  Accessories  01-07-2024         3   
1     44605   a5135ea0     Wireless Mouse  Electronics  18-01-2025         3   
2     44605   46a02a9b       Laptop Stand       Office  25-11-2024         5   
3     44605   f2f25eef  Bluetooth Speaker    Computers  31-07-2024         5   
4     44605   1777e8cb  Bluetooth Speaker    Computers  14-10-2024         4   

    price        country region marketing_channel  is_returned return_reason  
0  175.40  United States   West    Organic Search            1  Changed Mind  
1  151.70  United States  South          Referral            0           NaN  
2   52.10  United States  South          Referral            0           NaN  
3  158.03  United States   West          Referral            0           NaN  
4   24.51  United States  South    Organic Search            0           NaN  


In [3]:
# STEP 2: Clean and Transform Data
# Convert order_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# Drop rows with missing order_id or product_id
df.dropna(subset=['order_id','product_id'], inplace=True)

# Ensure is_returned is binary
df['is_returned'] = df['is_returned'].fillna(0).astype(int)


In [4]:
# STEP 3: Save to SQL (SQLite3)
conn = sqlite3.connect('product_orders.db')
df.to_sql('orders', conn, if_exists='replace', index=False)
print("Data loaded into SQLite database successfully.")

Data loaded into SQLite database successfully.


In [5]:
# STEP 4: SQL Queries for Return Rate
query = """
SELECT product_id, product_name, category, COUNT(*) AS total_orders,
SUM(CASE WHEN is_returned = 1 THEN 1 ELSE 0 END) AS total_returns,
ROUND(SUM(CASE WHEN is_returned = 1 THEN 1.0 ELSE 0 END) / COUNT(*), 2) AS return_rate
FROM orders
GROUP BY product_id, product_name, category
ORDER BY return_rate DESC;
"""
return_rate_df = pd.read_sql_query(query, conn)
print("Return rate summary:")
print(return_rate_df.head())

Return rate summary:
  product_id                 product_name     category  total_orders  \
0   00c59f30                 Laptop Stand  Accessories             1   
1   0534a101            Bluetooth Speaker  Accessories             1   
2   06a241af               Wireless Mouse    Computers             1   
3   095ffa81               Wireless Mouse       Office             1   
4   0b531932  Noise Cancelling Headphones  Accessories             1   

   total_returns  return_rate  
0              1          1.0  
1              1          1.0  
2              1          1.0  
3              1          1.0  
4              1          1.0  


In [10]:
# Calculate return percentage
query = """Select category, region, COUNT(*) AS total_orders,
SUM(is_returned) AS total_returns,
ROUND(SUM(is_returned) * 100.0 / COUNT(*), 2) AS return_percentage
FROM orders
GROUP BY category, region
ORDER BY return_percentage DESC;"""

return_percent_df = pd.read_sql_query(query, conn)
print("Return percentage summary:")
print(return_percent_df.head())

Return percentage summary:
      category     region  total_orders  total_returns  return_percentage
0  Accessories       West            32             10              31.25
1        Audio       West            32             10              31.25
2        Audio      South            29              9              31.03
3        Audio  Northeast            26              7              26.92
4    Computers  Northeast            23              6              26.09


In [11]:
# STEP 5: Save Cleaned & Aggregated Data for Power BI
return_rate_df.to_csv("return_rate_summary.csv", index=False)
df.to_csv("cleaned_product_orders.csv", index=False)
print("Cleaned and aggregated data exported as CSV for Power BI.")

Cleaned and aggregated data exported as CSV for Power BI.
